In [28]:
import pymongo 
from pymongo import MongoClient
import pandas as pd
import requests
import pandas as pd
%matplotlib inline

connection = MongoClient()
db = connection['espn_leagues_drafts']
standard_auction_10team_roto_picks = db['standard_auction_10team_roto_picks']
standard_snake_10team_roto_picks = db['standard_snake_10team_roto_picks']
auction_drafts = db['auction_drafts']

In [29]:
auctions_df = pd.DataFrame(list(standard_auction_10team_roto_picks.find()))
snakes_df = pd.DataFrame(list(standard_snake_10team_roto_picks.find()))

In [54]:
auction_counts = auctions_df.groupby('player_id').count()["_id"]
auction_counts = auction_counts[auction_counts>=100]
player_ids = list(auction_counts.index)

auctions_df_filtered = auctions_df[auctions_df['player_id'].isin(player_ids)] 
snakes_df_filtered = snakes_df[snakes_df['player_id'].isin(player_ids)] 

In [55]:
median_auction_price = auctions_df_filtered.groupby('player_id').median()['price']
mean_auction_price = auctions_df_filtered.groupby('player_id').mean()['price']

median_snake_pick = snakes_df_filtered.groupby('player_id').median()['pick_num']
mean_snake_pick = snakes_df_filtered.groupby('player_id').mean()['pick_num']

In [56]:
auction_summary = pd.concat([median_auction_price,median_auction_price.rank(ascending=False),
                             mean_auction_price,mean_auction_price.rank(ascending=False)], 
                             axis=1, join='inner')

auction_summary.columns = ['median_auction_price','median_auction_price_rank',
                           'mean_auction_price','mean_auction_price_rank']

In [57]:
snake_summary = pd.concat([median_snake_pick,median_snake_pick.rank(ascending=True),
                           mean_snake_pick,mean_snake_pick.rank(ascending=True)], 
                             axis=1, join='inner')

snake_summary.columns = ['median_snake_pick','median_snake_pick_rank',
                           'mean_snake_pick','mean_snake_pick_rank']


In [101]:
draft_summary = pd.concat([auction_summary,snake_summary],axis=1)
draft_summary['median_draft_type_diff'] = draft_summary['median_snake_pick_rank'] - draft_summary['median_auction_price_rank']
draft_summary['mean_draft_type_diff'] = draft_summary['mean_snake_pick_rank'] -  draft_summary['mean_auction_price_rank']
draft_summary.index = draft_summary.index.astype(int)

In [102]:
#all players, merged with prices!

In [103]:
espn_player_ranks = pd.DataFrame.from_csv('espn_auction_values/players_prices_with_ids.csv')
espn_player_ranks['espn_rank'] = espn_player_ranks.index +1
espn_player_ranks = espn_player_ranks.set_index('player_id')
all_players = list(db['master_players_list'].find())
all_players = [x['_id'] for x in all_players]
all_players = pd.DataFrame(all_players)
all_players['player_id'] = all_players['player_id'].astype(int)
all_players = all_players.set_index('player_id')
all_players = all_players.join(espn_player_ranks, lsuffix='all', rsuffix = 'espn').drop('nameespn', axis=1)

In [105]:
draft_summary.join(all_players)

,median_auction_price,median_auction_price_rank,mean_auction_price,mean_auction_price_rank,median_snake_pick,median_snake_pick_rank,mean_snake_pick,mean_snake_pick_rank,median_draft_type_diff,mean_draft_type_diff,nameall,pos,pos_rank,price_10,team,price_12,espn_rank
player_id,,,,,,,,,,,,,,,,,
1019,5,152.5,5.539548,150,141,141.5,135.492821,136,-11.0,-14,Yadier Molina,C,7,6,StL,8,145
1044,7,124.5,6.966895,123,122,119.5,121.888376,118,-5.0,-5,David Wright,3B,14,7,NYM,10,122
1060,2,198.5,2.482072,212,210,210.0,208.118460,217,11.5,5,Santiago Casilla,RP,34,1,SF,2,241
1083,1,272.5,1.475342,275,229,252.5,221.384037,254,-20.0,-21,Ryan Howard,1B,30,1,Phi,2,240
1103,1,272.5,1.594387,256,218,224.0,211.875069,223,-48.5,-33,Curtis Granderson,OF,51,2,NYM,5,190
1113,8,115.5,7.780468,119,119,117.5,117.476089,114,2.0,-5,Huston Street,RP,13,7,LAA,10,126
1123,1,272.5,1.933824,228,238,328.0,226.454545,296,55.5,68,Kendrys Morales,1B,32,0,KC,1,272
1125,1,272.5,1.200549,327,238,328.0,227.953521,304,55.5,-23,Aaron Hill,2B,21,0,Ari,1,255
1126,1,272.5,1.543081,268,236,316.0,229.926415,318,43.5,50,Brandon McCarthy,SP,68,1,LAD,2,248
